In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
plt.style.use('ggplot')
color_pal = ["#F8766D", "#D39200", "#93AA00", "#00BA38", "#00C19F", "#00B9E3", "#619CFF", "#DB72FB"]

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

In [2]:
df_raw = pd.read_csv('jan_1_zero.csv')
df_raw.drop("Unnamed: 0", axis=1, inplace=True)

## 주행거리가 0인데 요금이 존재하는 데이터
### 결측치로 가정하고 예측하는 모델을 생성한다.

1. Missing Values Imputation
2. RandomForest Regressor

## Missing Values Imputation

In [3]:
df = df_raw

In [4]:
# 주행거리가 0인데 요금이있는경우
idx_array = np.array(df[(df['trip_km']==0) & (df['trip_total']!=0)]['trip_km'].index)

In [5]:
df.iloc[idx_array,4] = np.nan

In [6]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(df)

X_test = 
# the model learns that the second feature is double the first
print(np.round(imp.transform(X_test)))

# Random Forest 

In [7]:
# 결측치 제거
df = df_raw
df.drop(['trip_start_timestamp', 'trip_end_timestamp', 'trip_miles', 'pickup_community_area', 'dropoff_community_area','payment_type'], axis=1, inplace=True)
df = df.dropna()
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)

In [8]:
df.__len__()

1384038

In [9]:
# 주행거리가 0인데 요금이있는경우
idx_array1 = np.array(df[(df['trip_km']==0)].index)
idx_array2 = np.array(df[(df['trip_km']!=0)].index)

In [10]:
df.iloc[idx_array1,:]

,taxi_id,trip_seconds,fare,tips,tolls,extras,trip_total,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_minuate,trip_km
4,7418.0,180.0,5.00,2.0,0.0,1.5,8.50,210.0,470.0,744.0,605.0,3.0,0.0
6,6641.0,420.0,7.25,0.0,0.0,0.0,7.25,419.0,615.0,411.0,545.0,7.0,0.0
8,7984.0,420.0,6.25,0.0,0.0,0.0,6.25,419.0,615.0,681.0,196.0,7.0,0.0
10,221.0,720.0,10.75,0.0,0.0,0.0,10.75,642.0,32.0,411.0,545.0,12.0,0.0
12,5367.0,420.0,6.75,0.0,0.0,0.0,6.75,158.0,270.0,158.0,270.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384008,5122.0,420.0,7.25,1.0,0.0,0.0,8.25,744.0,605.0,18.0,610.0,7.0,0.0
1384012,1592.0,480.0,10.00,0.0,0.0,0.0,10.00,599.0,346.0,199.0,510.0,8.0,0.0
1384017,7976.0,0.0,3.25,0.0,0.0,0.0,3.25,686.0,500.0,686.0,500.0,0.0,0.0
1384029,5910.0,300.0,5.75,1.5,0.0,0.0,7.25,18.0,610.0,30.0,405.0,5.0,0.0


In [11]:
# train, test, val set을 설정한다.
# validation set은 위의 조건에 맞는 주행거리가 0인데 요금 정보가 있는 값들로 한다.

# train, test set
df1 = df.iloc[idx_array2,:]

# train, test split
train_set, test_set = train_test_split(df1, test_size=0.3)

# validation set
val_set = df.iloc[idx_array1,:]

In [12]:
print(df.__len__(), df1.__len__(), val_set.__len__())

1384038 1095206 288832


In [13]:
X_cols = list(df.columns)
# 예측할 target 값인 trip_km
X_cols.remove('trip_km')

In [14]:
# Train, Test set split
X_train, X_test = train_set[X_cols], test_set[X_cols]
y_train, y_test = train_set['trip_km'], test_set['trip_km']
X_val, y_val = val_set[X_cols], val_set['trip_km']

# 모델생성

## 기본 옵션으로 모델 생성

In [15]:
rf_uncustomized = RandomForestRegressor()
rf_uncustomized.fit(X_train, y_train)
# Train 데이터 설명력
print("Score on training set : {:.3f}".format(rf_uncustomized.score(X_train,y_train)))
print("Score on test set : {:.3f}".format(rf_uncustomized.score(X_test,y_test)))

Score on training set : 0.985
Score on test set : 0.896


In [21]:
# 예측값 생성을 위한 validation set
X_val

,taxi_id,trip_seconds,fare,tips,tolls,extras,trip_total,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_minuate
4,7418.0,180.0,5.00,2.0,0.0,1.5,8.50,210.0,470.0,744.0,605.0,3.0
6,6641.0,420.0,7.25,0.0,0.0,0.0,7.25,419.0,615.0,411.0,545.0,7.0
8,7984.0,420.0,6.25,0.0,0.0,0.0,6.25,419.0,615.0,681.0,196.0,7.0
10,221.0,720.0,10.75,0.0,0.0,0.0,10.75,642.0,32.0,411.0,545.0,12.0
12,5367.0,420.0,6.75,0.0,0.0,0.0,6.75,158.0,270.0,158.0,270.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1384008,5122.0,420.0,7.25,1.0,0.0,0.0,8.25,744.0,605.0,18.0,610.0,7.0
1384012,1592.0,480.0,10.00,0.0,0.0,0.0,10.00,599.0,346.0,199.0,510.0,8.0
1384017,7976.0,0.0,3.25,0.0,0.0,0.0,3.25,686.0,500.0,686.0,500.0,0.0
1384029,5910.0,300.0,5.75,1.5,0.0,0.0,7.25,18.0,610.0,30.0,405.0,5.0


In [22]:
# 모델을 통한 예측값 생성
rf_uncustomized.predict(X_val)

array([1.014677, 2.191105, 1.622524, ..., 1.185597, 1.359883, 2.335157])

In [23]:
# 예측 값을 원본 데이터 프레임에 삽입
df.iloc[idx_array1,-1] = rf_uncustomized.predict(X_val)

In [24]:
# 주행거리가 0이었던 값들이 모델의 예측값으로 변경됨
df.iloc[idx_array1,-1]

4          1.014677
6          2.191105
8          1.622524
10         4.233866
12         2.138811
             ...   
1384008    2.164543
1384012    3.635516
1384017    1.185597
1384029    1.359883
1384030    2.335157
Name: trip_km, Length: 288832, dtype: float64

In [26]:
# 변경한 데이터 프레임에서 주행요금은 있지만 주행 거리가 없는 데이터 확인
df[(df['trip_km']==0.0) & (df['trip_total']!=0.0)].__len__()

0

In [ ]:
# 새로 변경한 데이터 프레임 저장
df.to_csv('new_jan_1.csv',index=False)